In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import hog
import os
import sys
sys.path.append("..")
from utils.detector import sliding_window
from utils.data import iou
from tqdm import tqdm

In [2]:
img_path = "../raw/image_kitti/"
label_path = "../raw/label_kitti/"

In [3]:
ignore = ["Pedestrian", "Person_sitting", "Cyclist", "Tram", "DontCare"]

In [4]:
def crop(img, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return img[ y1 : y2, x1: x2,:]

In [14]:
from math import ceil
label_dict = dict()
def to_int_array(array):
    return [ceil(float(x)) for x in array]
for img_label_name in os.listdir(label_path):
    index = img_label_name.split(".")[0]
    img_label_path = label_path + img_label_name
    with open(img_label_path, "r") as file:
        data = [line.strip().split(" ") for line in file.readlines()] 
        data = [ (entry[0], to_int_array(entry[4:8])) for entry in data if entry[0] not in ignore]
        label_dict[index] = data

In [15]:
label_dict['000001']

[('Truck', [600, 157, 630, 190]), ('Car', [388, 182, 424, 204])]

In [16]:
save_path = "test_case/"
window_size = (104, 56)

In [20]:
for index, img_name in tqdm(enumerate(os.listdir(img_path))):
    img = cv2.imread(img_path + img_name, cv2.IMREAD_UNCHANGED)
    img_index = img_name.split(".")[0]
    for i, entry in enumerate(label_dict[img_index]):
        box = entry[1]
        croped_img = crop(img, (box[0], box[1]), (box[2], box[3]))
        if croped_img.shape[1] < window_size[0] or croped_img.shape[0] < window_size[1]:
            continue
        resized = cv2.resize(croped_img, window_size, interpolation = cv2.INTER_AREA)
        cv2.imwrite(save_path + str(index) + "_" + str(i) + '.jpg', resized)

7481it [03:02, 40.95it/s]
